# Spinning Top (Kreisel) Simulation
This notebook simulates a spinning top modeled as a rigid triangle of masses connected to a fixed point.

In [ ]:
import sys
import os
import math
import time
from pythreejs import *
import numpy as np

# Adjust path to find the module
build_dir = "../build/mechsystem"
sys.path.insert(0, build_dir)

try:
    from mass_spring import *
except ImportError:
    print("Error: Could not import mass_spring module. Make sure you have built the project.")


## System Setup
We set up three masses forming a rigid triangle, with one mass connected to a fixed pivot.

In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# Fix at Origin
f1 = mss.add(Fix((0, 0, 0)))

# M1 at (0, 0, 1) - The "tip" (connected to Fix)
m1 = mss.add(Mass(1.0, (0, 0, 1)))

# M2 at (1, 0, 2)
m2 = mss.add(Mass(1.0, (1, 0, 2)))

# M3 at (0, 1, 2)
m3 = mss.add(Mass(1.0, (0, 1, 2)))

# Constraints
# 1. Pivot: Fix - M1
mss.add(DistanceConstraint(1.0, (f1, m1)))

# 2. Rigid Triangle: M1-M2, M2-M3, M3-M1
d = math.sqrt(2)
mss.add(DistanceConstraint(d, (m1, m2)))
mss.add(DistanceConstraint(d, (m2, m3)))
mss.add(DistanceConstraint(d, (m3, m1)))

print(f"Number of masses: {len(mss.masses)}")
print(f"Number of constraints: {len(mss.constraints)}")

Number of masses: 3
Number of constraints: 4


## Initial State
Setting initial velocities to create spin.

In [3]:
w = 10.0 # rad/s

# Create vectors for setState
n_masses = len(mss.masses)
dim = 3
x = Vector(dim * n_masses)
dx = Vector(dim * n_masses)
ddx = Vector(dim * n_masses)

# Fill x (positions)
for i, m in enumerate(mss.masses):
    p = m.pos
    x[3*i+0] = p[0]
    x[3*i+1] = p[1]
    x[3*i+2] = p[2]

# Fill dx (velocities)
# M1 (idx 0): (0,0,0)
dx[0] = 0; dx[1] = 0; dx[2] = 0

# M2 (idx 1): (0, w, 0)
dx[3] = 0; dx[4] = w; dx[5] = 0

# M3 (idx 2): (-w, 0, 0)
dx[6] = -w; dx[7] = 0; dx[8] = 0

# Fill ddx (accelerations) - 0
for i in range(dim * n_masses):
    ddx[i] = 0.0

# Set state
mss.setState(x, dx, ddx)
print("Initial state set.")

Initial state set.


## Visualization Setup
Using `pythreejs` to render the scene.

In [4]:
mass_meshes = []
for m in mss.masses:
    mass_meshes.append(
        Mesh(
            SphereBufferGeometry(0.1, 16, 16),
            MeshStandardMaterial(color="red"),
            position=m.pos,
        )
    )

fix_meshes = []
for f in mss.fixes:
    fix_meshes.append(
        Mesh(
            BoxBufferGeometry(0.2, 0.2, 0.2),
            MeshStandardMaterial(color="blue"),
            position=f.pos,
        )
    )

constraint_pos = []
for c in mss.constraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraint_pos.append([pA, pB])

constraint_geo = LineSegmentsGeometry(positions=constraint_pos)
constraint_mat = LineMaterial(linewidth=2, color="cyan")
constraints_line = LineSegments2(constraint_geo, constraint_mat)

view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[3, 3, 3], aspect=view_width/view_height, look_at=[0, 0, 1])
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight(intensity=0.5)

scene = Scene(children=[*mass_meshes, *fix_meshes, constraints_line, camera, key_light, ambient_light, AxesHelper(1)])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)
renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

## Simulation Loop
Running the simulation using `mss.simulate`.

In [5]:
# Simulation parameters
dt = 0.00001 # Small time step for stability
steps_per_frame = 1000 # 0.01s per frame

print("Starting simulation...")
for i in range(10000):
    mss.simulate(0.01, steps_per_frame)
    
    # Update visualization
    for m, mesh in zip(mss.masses, mass_meshes):
        mesh.position = tuple(m.pos)
        
    new_constraint_pos = []
    for c in mss.constraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        new_constraint_pos.append([pA, pB])
    
    constraints_line.geometry = LineSegmentsGeometry(positions=new_constraint_pos)
    
    time.sleep(0.01)

Starting simulation...


KeyboardInterrupt: 